<h1>LN Segmenting and Clustering Neighborhoods in the City of Toronto, Canada</h1>

<h3>Assignment Question 1</h3>
<h4>1.To install and import required library files</h4>
<h4>2.To link the portal url having postal codes of canada</h4>
<h4>3.To download the postcode table details from the url and convert the output to CSV file</h4>
<h4>4.To create a new dataframe from the CSV file</h4>
<h4>5.Cleaning the dataset</h4>
<h4>6.To exclude the rows where column Borough having Not Assigned Status</h4>
<h4>7.To map Borough details to neighborhood having Not Assigned Status</h4>
<h4>8.To merge Neighborhood into single row where unique postcode having multiple Neighborhoods</h4>
<h4>9.To display final output of the dataframe with three columns, Postcode, Borough, Neighborhood</h4>
<h4>10. To display the shape of the resultant dataframe</h4>


In [1]:
#import required library files
import pandas as pd
import numpy as np
import csv
import requests
import re
!pip install beautifulsoup4
from bs4 import BeautifulSoup
import urllib
from urllib.request import urlopen

<h3>The below statment will link the url, to fetch the table data from the url and finaly to perform cleaning activity to get the desired output</h3>

In [2]:

url="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
html = urlopen(url)
#soup = BeautifulSoup(html, 'lxml')
soup = BeautifulSoup(html)

htmtable=soup.find("table", {"class":"wikitable sortable"})

list_of_rows = []
for row in htmtable.findAll('tr'):
    list_of_cells = []
    for cell in row.findAll('td'):
        text = cell.text.replace('&nbsp;', '')
        text = cell.text.replace('\r\n', '')
        text = cell.text.strip()
        list_of_cells.append(text)
    list_of_rows.append(list_of_cells)
        

In [3]:
df=pd.DataFrame(list_of_rows)
df.head()

,0,1,2
0,None,None,None
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village


In [4]:
#To export to csv file from the dataframe df
df.to_csv('out2.csv',index=False)
df2=pd.read_csv('out2.csv',names=['Postcode', 'Borough', 'Neighbourhood'])
df2.head()

,Postcode,Borough,Neighbourhood
0,0,1,2
1,NaN,NaN,NaN
2,M1A,Not assigned,Not assigned
3,M2A,Not assigned,Not assigned
4,M3A,North York,Parkwoods


In [5]:
#to view the columns of the dataframe df2
df2.columns

Index(['Postcode', 'Borough', 'Neighbourhood'], dtype='object')

In [6]:
#to remove row zero from the dataframe df2
df2=df2.drop(index=0)
df2.head()

,Postcode,Borough,Neighbourhood
1,NaN,NaN,NaN
2,M1A,Not assigned,Not assigned
3,M2A,Not assigned,Not assigned
4,M3A,North York,Parkwoods
5,M4A,North York,Victoria Village


In [7]:
#to remove the NaN values from the dataframe df2
df2=df2.dropna()
df2.head()

,Postcode,Borough,Neighbourhood
2,M1A,Not assigned,Not assigned
3,M2A,Not assigned,Not assigned
4,M3A,North York,Parkwoods
5,M4A,North York,Victoria Village
6,M5A,Downtown Toronto,Harbourfront


In [8]:
#to check the shape of the dataframe df2
a_1=df2.shape[0]
print('1. Total Rows : ',a_1)

1. Total Rows :  288


<h3>To ignore the cells with a Borough that is Not Assigned </h3>

In [9]:
df3=df2[df2['Borough'] != 'Not assigned']
df3.head()

,Postcode,Borough,Neighbourhood
4,M3A,North York,Parkwoods
5,M4A,North York,Victoria Village
6,M5A,Downtown Toronto,Harbourfront
7,M5A,Downtown Toronto,Regent Park
8,M6A,North York,Lawrence Heights


In [10]:
#to check the shape of the dataframe post removing the not assigned values from Borough
a_2=df3.shape[0]
print('2. Total Rows : ',a_2)

2. Total Rows :  211


<h3>The below statments will perform the action where the neighborhood will be the same as the borough, if a cell has a borough but a Not assigned neighborhood</h3>

In [12]:
#to create a new list post performing the above statments
newopt1=[]
    
for rcnt in df3.index:

    a1=df3['Postcode'][rcnt]
    b1=df3['Borough'][rcnt]
    
    nbh=df3['Neighbourhood'][rcnt]
        
    if nbh != 'Not assigned': # to check whether neighborhood assigned
        c1=df3['Neighbourhood'][rcnt] #the vale remains the same
    else:
        C1=df3['Borough'][rcnt] #the value will be changed to borough
    
    fval=(a1,b1,c1)
    newopt1.append(fval)

In [13]:
#To copy the list to new dataframe df3
df4=pd.DataFrame(newopt1,columns=['Postcode', 'Borough', 'Neighbourhood'])
df4.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


In [14]:
#to verify the column values of the dataframe df4
df4.columns

Index(['Postcode', 'Borough', 'Neighbourhood'], dtype='object')

<h3>In the dataframe there are multiple neighborhood for single postcode, the same to be combined into one row with the neighborhoods separated with a comma</h3>

In [15]:
#to create a new list post performing the above statments

dfunq = df4['Postcode'].unique()

newpostcde=[]
ap=1
bb=1
cn=1

for pstcde in dfunq:
    p1=pstcde
    dfbrgh=df4[df4['Postcode'] == p1]
    ap=ap+1
    b1=" "
    for brgh in dfbrgh:
        dfb1=dfbrgh['Borough'].to_list()
        b1=dfb1[0]
        bb=bb+1
        dfnhod=df4[df4['Postcode'] == p1]
        dfn1=" "
        for nhod in dfnhod['Neighbourhood']:
            dfn1 = dfn1+nhod+','
            cn=cn+1
        c1=dfn1
    fval=(p1,b1,c1)
    newpostcde.append(fval)
    fval=" "
    
print('Total Rows : ',ap-1)

Total Rows :  103


In [18]:
df_final1=pd.DataFrame(newpostcde,columns=['Postcode', 'Borough','Neighbourhood'])
df_final1.head(10)

,Postcode,Borough,Neighbourhood
0,M3A,North York,"Parkwoods,"
1,M4A,North York,"Victoria Village,"
2,M5A,Downtown Toronto,"Harbourfront,Regent Park,"
3,M6A,North York,"Lawrence Heights,Lawrence Manor,"
4,M7A,Queen's Park,"Lawrence Manor,"
5,M9A,Etobicoke,"Islington Avenue,"
6,M1B,Scarborough,"Rouge,Malvern,"
7,M3B,North York,"Don Mills North,"
8,M4B,East York,"Woodbine Gardens,Parkview Hill,"
9,M5B,Downtown Toronto,"Ryerson,Garden District,"


In [19]:
df_final1.shape

(103, 3)

<h3>Assignment Question 2</h3>
<h3>To map the Geospatial_data csv file in the link to map the geographical coordinates of each postal code</h3>
<h4>1.To create a dataframe with the URL having Geo Coordinates details for each postcode</h4>
<h4>2.Cleaning the dataframe</h4>
<h4>3.Merging the postcode dataframe with Goe coordinates dataframe, adding Latitude and Longitude details</h4>
<h4>4.Display the final data frame with the column names : Postcode, Borough, Neighborhood, Latitude, Longitute </h4>
<h4>5.Display the shape of the final data frame </h4>

In [20]:
#to create the dataframe dfgeo using the url having latitude and longitute for each postcode
url1="https://cocl.us/Geospatial_data"
dfgeo=pd.read_csv(url1,names=['Postcode', 'Latitude', 'Longitude'])

In [21]:
dfgeo.head()

,Postcode,Latitude,Longitude
0,Postal Code,Latitude,Longitude
1,M1B,43.8066863,-79.1943534
2,M1C,43.7845351,-79.1604971
3,M1E,43.7635726,-79.1887115
4,M1G,43.7709921,-79.2169174


In [22]:
#to remove row zero having header information and to display the final output
dfgeo=dfgeo.drop(index=0)
dfgeo.head()

,Postcode,Latitude,Longitude
1,M1B,43.8066863,-79.1943534
2,M1C,43.7845351,-79.1604971
3,M1E,43.7635726,-79.1887115
4,M1G,43.7709921,-79.2169174
5,M1H,43.773136,-79.2394761


In [23]:
#to map the geo coordinates with the final ouput dataframe df_final1 and to create a new dataframe geo_result with all required columns 
geo_result = pd.merge(df_final1, dfgeo, how='left',on=['Postcode'])
geo_result.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,"Parkwoods,",43.7532586,-79.3296565
1,M4A,North York,"Victoria Village,",43.7258823,-79.3155716
2,M5A,Downtown Toronto,"Harbourfront,Regent Park,",43.6542599,-79.3606359
3,M6A,North York,"Lawrence Heights,Lawrence Manor,",43.718518,-79.4647633
4,M7A,Queen's Park,"Lawrence Manor,",43.6623015,-79.3894938


In [24]:
#to view the shape for the data frame geo_result
geo_result.shape

(103, 5)

<h3>Assignment Question 3</h3>
<h3>To visualize the Cluster in the neighborhoods of Toronto. To work with only boroughs that contain the word Toronto</h3>
<h4>1.Installation and importing of library files</h4>
<h4>2.To create a dataframe with the column Borough which contains the word Toronto</h4>
<h4>3.To create a basic map of Toronto, Canada </h4>
<h4>4.To display the map of Toronto with their neighborhood highlighted wih icons and labels</h4>
<h4>5.map of Toronto with their neighborhood highlighted only with labels </h4>
<h4>6.map of Toronto displaying neighborhoods with Circle Marks and showing their labels </h4>
<h4>7.Toronto Map showing the CLUSTERS and displaying the Neighborhoods for each CLUSTER </h4>

In [25]:
#to install and import the required files
!pip install folium

import folium
import pandas as pd
import matplotlib
%matplotlib inline

print('Folium installed and imported!')

     |████████████████████████████████| 92kB 13.5MB/s eta 0:00:01
Folium installed and imported!


<h4>To create a dataframe, to work with only Borough that contains the word Toronto</h4>

In [29]:
#to create a dataframe df_incidents post filtering the columnn Borough which contains the word Toronto
df_incidents=geo_result[geo_result['Borough'].str.contains("Toronto") == True]
df_incidents.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
2,M5A,Downtown Toronto,"Harbourfront,Regent Park,",43.6542599,-79.3606359
9,M5B,Downtown Toronto,"Ryerson,Garden District,",43.6571618,-79.3789371
15,M5C,Downtown Toronto,"St. James Town,",43.6514939,-79.3754179
19,M4E,East Toronto,"The Beaches,",43.6763574,-79.2930312
20,M5E,Downtown Toronto,"Berczy Park,",43.6447708,-79.3733064


<h4>To create the map of Toronto, Canada</h4>

In [26]:
#to generate the map of toronto,canada
tlatitude=43.651070
tlongitude=-79.347015

toronto_map = folium.Map(location=[tlatitude,tlongitude], zoom_start=12)
toronto_map

<h4>Method 1: To create the map of Toronto, Canada with marking the neighborhood showing their label and icon</h4>

In [30]:
import pandas as pd
import folium
from folium.features import CustomIcon
from folium.plugins import MarkerCluster
from folium import plugins

#assign the long and lat values
tlatitude=43.651070
tlongitude=-79.347015

#create the map
toronto_map = folium.Map(location=[tlatitude,tlongitude], zoom_start=12)

# instantiate a feature group for the incidents in the dataframe
incidents = folium.map.FeatureGroup()
latitudes = list(df_incidents.Latitude)
longitudes = list(df_incidents.Longitude)
labels = list(df_incidents.Neighbourhood)

for lat, lng, label in zip(latitudes, longitudes, labels):
    folium.Marker(
      location = [lat, lng], 
      popup = label,
      icon = folium.Icon(color='green', icon='info-sign')
     ).add_to(incidents)

#Add Incidents to the Map:
toronto_map.add_child(incidents)

#display the map
toronto_map

<h4>Method 2: To create the map of Toronto, Canada with marking the neighborhood with only labels</h4>

In [31]:
tlatitude=43.651070
tlongitude=-79.347015
toronto_map = folium.Map(location=[tlatitude,tlongitude], zoom_start=12)

# loop through the dataframe and add each data point to the mark cluster
for lat, lng, label, in zip(df_incidents.Latitude, df_incidents.Longitude, df_incidents.Neighbourhood):
    folium.Marker(location=[lat, lng],icon=None,popup=label,).add_to(incidents)

# add incidents to map
toronto_map.add_child(incidents)


# show map
toronto_map

<h4>Method 3: Map of Toronto,Canada with Neighborhoods location marked with CIRCLE marker with label</h4>

In [33]:
#Map with location marked with circle markers

tlatitude=43.651070
tlongitude=-79.347015

# create map and display it
toronto_map = folium.Map(location=[tlatitude,tlongitude], zoom_start=12)

for lat, lng, label in zip(df_incidents.Latitude, df_incidents.Longitude, df_incidents.Neighbourhood):
    folium.CircleMarker([lat, lng],radius=5, color='yellow', fill=True, popup=label,fill_color='blue',fill_opacity=0.6).add_to(toronto_map)

# show map
toronto_map

<h4>Toronto CLUSTERING the Neighborhood with different clusters</h4>

In [34]:
#To group the markers into different clusters and cluster to represented the number of neighborhood
                           
from folium import plugins

tlatitude=43.651070
tlongitude=-79.347015

# create map
toronto_map = folium.Map(location=[tlatitude,tlongitude], zoom_start=12)

# create mark cluster object
incidents = plugins.MarkerCluster().add_to(toronto_map)

for lat, lng, label, in zip(df_incidents.Latitude, df_incidents.Longitude, df_incidents.Neighbourhood):
    folium.Marker(location=[lat, lng],icon=None,popup=label,).add_to(incidents)

# add incidents to map
toronto_map.add_child(incidents)
    
# display map
toronto_map